In [8]:
from subprocess import PIPE
import numpy as np
from IPython.display import Image

# Ipython notebook tips

Some small tips of the new interface in python/ipython:

* Tab inside brackets gives the help of the command;
* or using:

In [2]:
from grass import script

Test the IPython API completition to access GRASS script API, uncomment the line below and press **```tab```** key 

In [3]:
#script.

Access IPython helper function using the **```?```** suffix

In [4]:
script?

# Replace BASH

GRASS6 and GRASS7, the traditional way for scripting.

Import grass script module, which has some useful functions:

In [1]:
from grass import script

Let's have a look which function containing the word command are available, with:

In [2]:
from grass.pygrass.utils import looking

looking(script, '*command*')

['exec_command',
 'feed_command',
 'get_commands',
 'get_real_command',
 'make_command',
 'parse_command',
 'pipe_command',
 'read_command',
 'run_command',
 'start_command',
 'write_command']

In [3]:
script.list_pairs(type='rast')

[(u'aaa_1', u'user1'),
 (u'aaa_1_rel', u'user1'),
 (u'aaa_2', u'user1'),
 (u'aaa_2_rel', u'user1'),
 (u'aaa_3', u'user1'),
 (u'aaa_3_rel', u'user1'),
 (u'aaa_4', u'user1'),
 (u'aaa_4_rel', u'user1'),
 (u'asp', u'user1'),
 (u'aspect', u'user1'),
 (u'basins', u'PERMANENT'),
 (u'doubled_elev', u'user1'),
 (u'elev_reclass', u'user1'),
 (u'elev_smoothed', u'user1'),
 (u'elev_strip', u'user1'),
 (u'elevation', u'PERMANENT'),
 (u'elevation100', u'PERMANENT'),
 (u'elevation100', u'user1'),
 (u'elevation100_new', u'user1'),
 (u'elevation_shade', u'PERMANENT'),
 (u'geology', u'PERMANENT'),
 (u'geology_on_basins', u'user1'),
 (u'hospitals', u'user1'),
 (u'lakes', u'PERMANENT'),
 (u'lakes_individual', u'user1'),
 (u'landuse', u'PERMANENT'),
 (u'newele', u'user1'),
 (u'newele2', u'user1'),
 (u'newele3', u'user1'),
 (u'newscratch', u'user1'),
 (u'prova_grad', u'user1'),
 (u'prova_gradient', u'user1'),
 (u'prova_gradient_inverse', u'user1'),
 (u'rgradient', u'user1'),
 (u'slope', u'user1'),
 (u'slope

### Start and wait until the end of the process

In [4]:
script.run_command('r.info', map='elevation')

0

### Parse the kwargs and return a list of parameters

In [5]:
script.make_command('r.info', map='elevation', flags='r')

[u'r.info', u'-r', u'map=elevation']

### Start and return a `Popen` object

In [6]:
script.start_command('r.info', map='elevation')

In [9]:
process = script.start_command('r.info', map='elevation', flags='r', stdout=PIPE)

In [10]:
stdout = process.stdout

In [11]:
print([line.strip().split('=') for line in stdout])

[['min', '55.57879'], ['max', '156.3299']]


### Parse the output of a command

In [ ]:
script.parse_command('r.info', map='elevation', flags='g', delimiter='=')

In [ ]:
script.parse_command('g.region', flags='p', delimiter=':')

### Catch the command output

In [ ]:
region = script.pipe_command('r.info', map='elevation', flags='r')
region

In [ ]:
stdout = ''.join(line for line in region.stdout)
print(stdout)

### Catch and return the stdout

In [ ]:
stdout = script.read_command('r.info', map='elevation', flags='r')
print(stdout)

In [ ]:
mrange = dict([line.split('=') for line in stdout.split('\n') if line != ''])
for k in mrange:
    mrange[k] = float(mrange[k])
mrange

### Use the pipe as input for another command

Write the rules that we want to use for the reclassification:

In [ ]:
keys = ['low', 'medium low', 'medium', 'medium high', 'high']
vals = np.linspace(mrange['min']-1, mrange['max']+1, num=len(keys)+1, endpoint=True)

rvals = [(int(vals[i-1])+1, int(vals[i]), i, keys[i-1]) for i in range(1, len(vals))]
rules = '\n'.join(['%3d thru %3d = %2d %s' % v for v in rvals])
print(rules)

Now we can pass as input for the r.reclass the rules using the function `write_command`:

In [ ]:
script.write_command('r.reclass', input='elevation', output='elev_reclass', rules='-', overwrite=True, stdin=rules)

### Display the map inside the ipython notebook

Start a new virtual monitor that generate a file: `view.png` that we can display inside the ipython notebook.

In [ ]:
script.run_command('d.mon', start='png', output='view.png', overwrite=True)

In [ ]:
script.run_command('d.rast', map='elevation')
Image('view.png')

In [ ]:
script.run_command('d.rast', map='elev_reclass')
Image('view.png')

In [ ]:
script.run_command('d.mon', stop='png')

# Summary

We have seen:

* how we can replace Bash script with Python;
* how to use some IPython Notebook features with GRASS Python;

# Exercise

## Time for coding!

Transform the following Bash script:

into a Python one:

## Create a GRASS module using scripting library

In [ ]:
%%file mygrassmodule.py
#!/usr/bin/env python
# -- coding: utf-8 --
#
############################################################################
#
# MODULE:	    <name of your module>
#
# AUTHOR(S):    <your name>
#
# COPYRIGHT:	(C) 2013 by the GRASS Development Team
#
#		This program is free software under the GNU General Public
#		License (>=v2). Read the file COPYING that comes with GRASS
#		for details.
#
#############################################################################

##-----------------------
## DESCRIPTION
##-----------------------
#%Module
#%  description: <description>
#%  keywords: <keyword 1>
#%  keywords: <keyword 1>
#%  keywords: <keyword 1>
#%  overwrite: yes
#%End

##-----------------------
## VECTOR INPUT
##-----------------------
#%option G_OPT_V_MAP
#%  key: key0
#%  description: <parameter description>
#%  required: yes
#%end

##-----------------------
## MULTI RASTER INPUT
##-----------------------
#%option G_OPT_R_INPUTS
#%  key: key1
#%  description: <parameter description>
#%  multiple: yes
#%  required: no
#%end

##-----------------------
## STRING INPUT
##-----------------------
#%option
#%  key: key2
#%  description: <parameter description>
#%  type: string
#%  multiple: yes
#%  required: no
#%end
from pprint import pprint
from grass.script import parser

def main(opts, flgs):
    print('OPTIONS:')
    pprint(opts)
    print('FLAGS:')
    pprint(flgs)

if __name__ == "__main__":
    opts, flgs = parser()
    main(opts, flgs)



In [ ]:
!python2 mygrassmodule.py --h

In [ ]:
!python2 mygrassmodule.py key0=pippo key1=pluto,topolino key2=minni,clarabella